## Imports & User-defined variables

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
#this may take 20-30 seconds
import torch
import glob
import re
import os
import math
import shutil
import pickle
import pandas as pd
import numpy as np
import gemmi
import reciprocalspaceship as rs
import matplotlib.pyplot as plt
from random import sample
from tqdm import tqdm
from sklearn import metrics
from multiprocessing import Pool
from itertools import repeat

import valdo

pd.set_option("display.precision", 3)
plt.rc('figure', figsize=(4,2.5))
print("done with imports.")

In [ ]:
# ncpu=1

In [ ]:
bGPU, ncpu=valdo.helper.configure_session()

### Set user-defined variables

In [ ]:
random_seed       = 11231        # random seed for PyTorch and subsampling data (if any) 
run_prefix        = "run_6_iso_" + str(random_seed) + "_"

# filtering input to MTZ
ref_mtz           = '058.mtz'    # use a high-quality dataset as reference
mtzs_to_ignore    = []           # MTZs to disregard
filter_by_Rfree   = True         # whether to filter out datasets for which refinement was poor (before scaling)
max_R_factor      = 0.4          # Worst apo refinement Rfree-factor to keep. otherwise discarded.
filter_by_cc      = True         # whether to filter out datasets fwith poor correlation with the reference (after scaling)
min_cc_ref        = 0.4

# VAE (most settings hardcoded below)
fraction          = 1.0          # fraction of datasets to work with (default: 1.0)
train_fraction    = 0.8          # fraction of working datasets used for training (sensible default: 0.8)
latent_dim        = 7            # VAE latent space dimension (sensible default: 7)
epochs            = 500          # Number of VAE training epochs (sensible default: 500)
include_errors    = True         # whether to use SIGFs in the VAE loss function (sensible default: True)
stdof             = 32         # DOF for student t (None -> normal dist); only used when include_errors=True
eps               = 0.02         # the smaller eps (>=0), the more strongly estimated errors are taken into account (sensible default: 0.01)

# difference maps
ml_recon          = True        # Whether to use the ML reconstruction from the VAE rather than a sample (default: False)
vae_samples       = 1            # number of samples to draw from the VAE during reconstruction (sensible default: 1), the variation is usually negligible
w_pcts            = (90.0,99.99) # percentiles on sigDF and DelDF used in weights calc (sensible default: 95.00, 99.99)
low_res_cutoff    = 1000.        # cutoff in A specifying what low-res reflections to keep (prob not helpful! sensible def: 1000.0)

# blob finding
blob_sig_cutoff   = 3.5          # cutoff for blob search (our default: 3.5)
radius_in_A       = 4.0          # radius (in A) of the gaussian smoothing kernel; 3x the sigma (default: 5)

In [ ]:
# Seed the random number generator for reproducible behavior
# change the random seed for another realization of the optimization.
torch.manual_seed(random_seed)
np.random.seed(random_seed)

### Define Paths

In [ ]:
# use trailing slashes!
my_dir             = "/n/holyscratch01/hekstra_lab/dhekstra/valdo-vatD/" # directory where to keep all the processing steps
original_data_path = "/n/hekstra_lab/projects/valdo_vatD_FraserLab/VatD/vatd_mtzs/" # the MTZs with measured F and sigF

# keep the logs and pdb files in the same directory, e.g. with names xxxx.pdb, xxxx.mtz, and xxxx.log
refinement_paths=["/n/hekstra_lab/projects/valdo_vatD_FraserLab/VatD/PHENIX_autorefinement/refine_output/short_names/"
                 ]
refinement_phases = [("PH2FOFCWT","PHFOFCWT")
                    ]
# PHENIX phase options: *PHIF-model, PHFC, PH2FOFCWT, PHFOFCWT
# dimple phase options: PHIC, PHIC_ALL, PHWT, *PHIC_ALL_LS

# we may need a parser map our data MTZs to the refinement MTZs. 
# here are some examples:
def apo_phases_parser_default(file): # for 0,2
    return os.path.basename(file)

def apo_phases_parser_4digit(file): # for 1
    return os.path.basename(file)[0:4]+".mtz"
    
def apo_phases_parser_dimple(file): # for 3
    return "dimple_"+os.path.basename(file)[0:4]+"/final.mtz"


apo_phases_parser = apo_phases_parser_default

which_phases = 0
refined_path = refinement_paths[which_phases] # points to folder with refinement results; used for blob finding and indexing ambiguation
phasing_path = refinement_paths[which_phases] # used for phasing of difference maps; could be the same as refined_path

phase_2FOFC_col_in  = refinement_phases[which_phases][0]
phase_FOFC_col_in   = refinement_phases[which_phases][1]
phase_2FOFC_col_out = 'refine_PH2FOFCWT'
phase_FOFC_col_out  = 'refine_PHFOFCWT'

In [ ]:
# general pattern for standardizing mtz filenames
mtz_file_pattern = r".*(\d{3}).*.mtz"

basepath = my_dir + 'pipeline/'
data_path      = basepath + 'data/'
input_mtz_path = basepath + 'data/mtzs_origin/'
reindexed_path = basepath + 'data/mtzs_reindex/'
scaled_path    = basepath + 'data/mtzs_scaled/'

# These currently do not carry prefixes bc they will mostly be constant:
intersection_path = scaled_path + 'intersection.pkl'
union_path        = scaled_path + 'union.pkl'
sigF_path         = scaled_path + 'sigF.pkl'

vae_path = basepath + 'vae/'
vae_reconstructed_path             = vae_path + 'reconstructed/'
vae_reconstructed_with_phases_path = vae_path + 'reconstructed_w_phases/'
blob_path                          = vae_path + 'blobs/'
bound_models_standardized_path= basepath + 'data/bound_models_reindexed/'

In [ ]:
amplitude_col        = 'FP' #'F-obs'
amplitude_scaled_col = amplitude_col + '-scaled'
error_col            = 'SIGFP' #'SIGF-obs'
error_scaled_col     = error_col + '-scaled'

#### Configure paths

In [ ]:
# Make paths
dir_hierarchy=[basepath, data_path, input_mtz_path, reindexed_path, refined_path, scaled_path, vae_path, vae_reconstructed_path, \
               vae_reconstructed_with_phases_path, blob_path, bound_models_standardized_path]
for folder in dir_hierarchy:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"{folder:<80}" + f"{'created': >20}")
    else:
        print(f"{folder:<80}" + f"{'already exists': >20}")

## Step 1: Diffraction Data

The first step involves acquiring diffraction datasets in the `mtz` format. These datasets should follow a specific naming convention, where each file is named with a number followed by the `.mtz` extension (e.g., `0001.mtz`, `0002.mtz`, etc.).

### Usage

1. Ensure that you have collected diffraction datasets in the `mtz` format.

2. Organize the datasets with sequential numerical names (e.g., `0001.mtz`, `0002.mtz`, etc.). You can do so using the following cell.

3. If you have bound-state models already and you want to benchmark against those in Step 9 below, also run the cell `Application to bound state models (for Step 9)`. 

Following this naming convention will allow datasets to be ready for further processing.

### Copy and Renaming Code
_Only need to run the following cell once, can skip on rerun._

The following cell is a template for renaming files to the correct naming convention. Change `source_path`, `destination_path`, and extensions as necessary.

e.g. `source_path/PTP1B-y0798_mrflagsref_idxs.mtz` -> `destination_path/0798.mtz`

In [ ]:
%%time
new_mtzs=valdo.helper.standardize_input_mtzs(source_path=original_data_path, 
                       destination_path=input_mtz_path, 
                       mtz_file_pattern=mtz_file_pattern, 
                       ncpu=ncpu)
print("\nCreated " + str(len(new_mtzs)) + " standardized MTZ files.")

## Step 2: Reindexing, Automatic Refinement & Scaling

This step focuses on automatic refinement and scaling a list of input MTZ files to a reference MTZ file using gemmi. If you have already performed automated refinement against each dataset, you can skip it here and just take this as information. The VatD data do not require reindexing. For an example that does require reindexing, see the PTP1B notebook.

**Scaling**: the present example then goes on to scaling in order to make sure the data have similar magnitude and resolution dependence before being fed into the VAE.
If only intensities (and not amplitudes) are available, you can perform French-Wilson scaling here too.

### Usage

The following cells do the following (and you'd need to do the same in a command script):

1. Import the required library, `valdo`.

2. If necessary, perform French-Wilson scaling (we recommend generating amplitudes using your data reduction software!).

3. Create a `Scaler` object by providing the path to the reference MTZ file.

4. Call the `batch_scaling()` method of the `Scaler` object. The `batch_scaling()` method will apply the scaling process to each input MTZ file and save the scaled MTZ files in the specified output folder. Scaling metrics, such as least squares values and correlations, will be saved in the report file.

    This function can be called with the following parameters:
    - `mtz_path_list`: List of paths to input MTZ files to be scaled.
    - `outputmtz_path`: Path to the folder where the scaled MTZ files will be saved (optional, default is `./scaled_mtzs/`).
    - `reportfile`: Path to the file where scaling metrics will be saved (optional, default is `./scaling_data.json`).
    - `verbose`: Whether to display verbose information during scaling (optional, default is `True`).
    - `n_iter`: Number of iterations for the analytical scaling method (optional, default is `5`).

### French-Wilson scaling (_if needed_) 
_Only need to run the following once, so can skip on rerun._

_Only run the two cells below if the data reduction software did not calculate suitable F, SigF._

In [ ]:
# List of files to be reindexed
file_list = glob.glob(input_mtz_path + "*mtz")
file_list.sort()

# Drop mtzs in the ignore list
for mtz in mtzs_to_ignore:
    file_list.remove(input_mtz_path+mtz)
print("Working with " + str(len(file_list)) + " MTZ files.")

In [ ]:
%%time
# if this does not complete within 5 min, please lower tmp_cpu below ncpu and try again.
tmp_cpu=12
tmp=valdo.helper.apply_french_wilson(file_list, \
                                     intensity_key="IMEAN", \
                                     sigma_key="SIGIMEAN", \
                                     output_columns=["IP","SIGIP","FP","SIGFP"], 
                                     isotropic=True, \
                                     ncpu=tmp_cpu
                                    )
print(f"Performed French-Wilson scaling for {np.sum(tmp)} out of {len(file_list)} input MTZ files.")

Select a good dataset as your reference (see the variable `ref_mtz` above).

**Critical**: The phases we will use below to calculate difference maps must come from models that have been indexed _consistent_ with the indexing convention chosen here (by the choice of our reference dataset). Otherwise, the difference maps will be meaningless!! An easy way to assure consistency is by obtaining the phases from refinement against the reindexed data!

### Automatic Refinement Code Example

Automatic refinement should happen outside the notebook. **Note:** It is recommended to run the automatic refinement in a parallel manner for many datasets.

In [ ]:
print("Example command to run automatic refinement:")
print(" ")
print(f"valdo.refine --pdbpath 'xxx/xxx_apo.pdb' --mtzpath '{input_mtz_path}*.mtz' --output '{refined_path}' --eff 'xxx/refine_drug.eff'\n")

### After Automatic refinement
After completing the refinement process, all outcomes will be located in the `refined_path` (which you can set in the first section of this notebook).
If you are utilizing PHENIX for refinement, you can generate refinement statistics using the following code. We will look at this because it may (or may not) be a good idea to filter your datasets based on how well they refine.

In [ ]:
loglist = glob.glob(os.path.join(refined_path, "*log"))
loglist.sort()

idx = []
Rw_start = []
Rf_start = []
Rw_final = []
Rf_final = []
time = []

print(os.path.basename(loglist[0]))
print(os.path.basename(loglist[0]).replace('.','_').split("_")) 

In [ ]:
for file in tqdm(loglist):
    with open(file, 'r') as f:
        log = f.readlines()

    # may need to modify the parsing of the filenames depending on your naming conventions:
    # print(os.path.basename(file).split("_")) 
    idx_temp, _  = os.path.basename(file).replace('.','_').split("_")
    try:
        Rw_final_temp, Rf_final_temp = float(log[-2].replace(","," ").split(" ")[3]), float(log[-2].replace(","," ").split(" ")[-1]) # Final
        Rw_start_temp, Rf_start_temp = float(log[-3].replace(","," ").split(" ")[3]), float(log[-3].replace(","," ").split(" ")[-1]) # Start
        time_temp = float(log[-1].strip().split(" ")[-2]) # Time
    except Exception as e:
        print("Sample ", idx_temp, " refinement fails!")
        print(e)
        Rw_final_temp, Rf_final_temp = float("nan"), float("nan")
        Rw_start_temp, Rf_start_temp = float("nan"), float("nan")
        time_temp = float("nan")
    idx.append(idx_temp)
    Rw_start.append(Rw_start_temp)
    Rf_start.append(Rf_start_temp)
    Rw_final.append(Rw_final_temp)
    Rf_final.append(Rf_final_temp)
    time.append(time_temp)

df_refine = pd.DataFrame({
    "file_idx"   : idx,
    "Rw_start"   : Rw_start,
    "Rf_start"   : Rf_start,
    "Rw_final"   : Rw_final,
    "Rf_final"   : Rf_final,
    "time(s)"    : time
})

In [ ]:
df_refine.head(5)

In [ ]:
df_badR = df_refine[df_refine["Rf_final"] > max_R_factor].copy()

In [ ]:
filter_list=[]
if filter_by_Rfree:
    # just keeping track. not yet applied!
    filter_list = (df_badR["file_idx"] + '.mtz').tolist()
    

### Scaling Code
_Only need to run the following cell once, can skip on rerun._

In [ ]:
file_list = glob.glob(input_mtz_path + "*mtz")
file_list.sort()

for mtz in filter_list:
    try:
        file_list.remove(input_mtz_path+mtz)
    except:
        print(mtz + " already not in list")
    
print("We have " + str(len(file_list)) + " MTZ files ready for scaling.")

The following optimization of scales sometimes generates numerical warnings about overflows. Let us know if this happens a lot. Without parallelization, this takes about 0.25s/dataset.

In [ ]:
reference_idx = file_list.index(input_mtz_path+ref_mtz)

In [ ]:
print(scaled_path)
# print(glob.glob(scaled_path + "*mtz"))
prior_scaled_mtzs=glob.glob(scaled_path + "*mtz")
print(f"Contained " +str(len(prior_scaled_mtzs)) + " files.")
print("Removing...")
for f in prior_scaled_mtzs:
    os.remove(f)
print("Done.")

In [ ]:
%%time
# Scales all datasets to the previously provided reference, writes a `metrics.pkl`, take ~30s
# Returns some metrics on the quality of the dataset.

if ncpu > 1:
    scaler = valdo.Scaler_pool(reference_mtz=file_list[reference_idx],columns=[amplitude_col, error_col],verbose=False, n_iter=5,ncpu=ncpu)
    scaling_metrics = scaler.batch_scaling(mtz_path_list=file_list, 
                                   outputmtz_path=scaled_path,
                                   prefix=run_prefix)
else:
    scaler = valdo.Scaler(reference_mtz=file_list[reference_idx])
    scaling_metrics = scaler.batch_scaling(mtz_path_list=file_list, 
                                   outputmtz_path=scaled_path,
                                   prefix=run_prefix, 
                                   verbose=False)

In [ ]:
scaling_metrics.head()

- Most of the datasets for which the LS goes up also have bad Rfree.
- Keep in mind that LS scales with number of reflections. A better comparison would normalize for that.

In [ ]:
plt.plot(scaling_metrics["start_LS"].to_numpy(),scaling_metrics["end_LS"].to_numpy(),'.',alpha=0.5)
xlim=plt.xlim()
plt.plot(xlim,xlim,'r-')
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Starting LS")
plt.ylabel("Final LS")
plt.grid()
plt.show()

plt.hist(scaling_metrics['end_corr'],20)
plt.xlabel('Correlation to reference')
plt.ylabel('Count')
plt.grid()
plt.show()

## Step 3: Normalization & Preparation of VAE input

This step involves normalizing the scaled structure factor amplitudes obtained in the previous step. The input is restricted to only those Miller indices present in the _intersection_ of all datasets, and the VAE will predicts structure factor amplitudes for all Miller indices in the _union_ of all datasets.

Additionally, we standardize all the input data, such that the structure factor amplitudes for each Miller index in the union of all datasets have a mean of zero and a unit variance across datasets. 

### Usage

1. Import the required library, `valdo.preprocessing`.

2. Find the intersection and union of the scaled datasets using the following functions:

   - `find_intersection()`: Finds the intersection of `amplitude_col` from multiple input MTZ files and saves the result to the specified output pickle file. Arguments include the following:

      - `input_files`: List of input MTZ file paths.
      - `output_path`: Path to save the output pickle file containing the intersection data.
      - `amplitude_col`: Name of the column in the dataset that represents the scaled amplitude (default is 'F-obs-scaled').

   - `find_union()`: Finds the union of `amplitude_col` from multiple input MTZ files and saves the result to the specified output pickle file. Arguments are the same as `find_intersection()`.

3. Generate the VAE input and output data using the `generate_vae_io()` function. This standardizes the intersection dataset using mean and standard deviation calculated from the union dataset. The standardized intersection becomes the VAE input, while the standardized union becomes the VAE output. Both the VAE input and output are saved to the specified folder. 

    This function can be called with the following parameters:

    - `intersection_path`: Path to the intersection dataset pickle file.
    - `union_path`: Path to the union dataset pickle file.
    - `io_path`: Path to the output folder where the VAE input and output will be saved. Mean and standard deviation data calculated from the union dataset will also be saved in this folder as `union_mean.pkl` and `union_sd.pkl`.

### Filtering Samples Code

_Only need to run this code once, can skip on rerun._

In this example, we remove samples with low `end_corr`. This ensures that our VAE is trained with high quality samples.

In [ ]:
# Identify all scaled files to use as input and output for the VAE

file_list = glob.glob(scaled_path + "*mtz")
file_list.sort()
print("Current file list contains " + str(len(file_list)) + " entries.")

This following cell removes samples with `end_corr < min_cc_ref` or if `end_corr = NA`.

In [ ]:
metrics_df = pd.read_pickle(scaled_path + run_prefix + "scaling_metrics.pkl")
# metrics_df[metrics_df.isna().any(axis=1)]

if filter_by_cc:
    low_corr_files = list(metrics_df[ (metrics_df['end_corr'] < min_cc_ref) | \
                                      (metrics_df['end_corr'].isnull())]['file'])
    low_corr_files = [scaled_path + x + '.mtz' for x in low_corr_files]
    file_list = [file for file in file_list if file not in low_corr_files]

with open(os.path.join(vae_path, 'filtered_file_list.pkl'), 'wb') as f:
    pickle.dump(file_list, f)
print("The file list contains " + str(len(file_list)) + " entries after filtering.")

##### For testing purposes, we will subsample the list of entries.

In [ ]:
if fraction < 1.0:
    file_list = sample(file_list, int(fraction * len(file_list)))
print("Subsampling " + str(len(file_list)) + " entries.")

### Generating VAE Input

The following cells generate the VAE input and output. These cells do take seconds to minutes to run (depending on `ncpu`)

**Please be patient!**

If you are not changing anything about scaling or dataset filtering, you can skip them next time!

In [ ]:
%%time
# Creates an `intersection.pkl` file at the specified path
# This is the intersection of all the scaled files provided
valdo.preprocessing.find_intersection(input_files=file_list, 
                  output_path=intersection_path,
                  amplitude_col=amplitude_scaled_col)

In [ ]:
%%time
# Creates a `union.mtz` file at the specified path
# This is the union of all the scaled files provided
# This is a lot slower than the intersection step about. The step is 
#   about linear in the number of files, 0.5 sec/file on my CPU.
# Inclusion of errors does not affect speed -- *always include them*

valdo.preprocessing.find_union(input_files=file_list, 
                               output_path=union_path,
                               sigF_path=sigF_path,
                               amplitude_col=amplitude_scaled_col, 
                               error_col=error_scaled_col,
                               include_errors=True)

In [ ]:
%%time
# Generates VAE input and output data from the intersection and union datasets
# Inclusion of errors does not affect speed => always include them here.
# Note that we do not currently prefix the VAE input
valdo.preprocessing.generate_vae_io(intersection_path=intersection_path, 
                                    union_path=union_path, 
                                    sigF_path=sigF_path,
                                    io_folder=vae_path, 
                                    prefix="", 
                                    include_errors=True)

## Step 4: VAE Training

In this step, we train the VAE model using the provided VAE class.

### Usage

1. Load the VAE input and output data that was generated in the previous step.

2. Initialize the VAE model with the desired hyperparameters. Tune-able hyperparameters include the following:
    - `n_dim_latent`: Number of dimensionality in latent space (optional, default `1`)

    - `n_hidden_layers`: Number of hidden layers in the encoder and decoder. If an int is given, it will applied to both encoder and decoder; If a length 2 list is given, first int will be used for encoder, the second will be used for decoder

    - `n_hidden_size`: Number of units in hidden layers. If an int is given, it will be applied to all hidden layers in both encoder and decoder; otherwise, an array with length equal to the number of hidden layers can be given, the number of units will be assigned accordingly.

    - `activation` : Activation function for the hidden layers (optional, default `tanh`)

3. Split the data into training and validation sets. Randomly select a subset of indices for training and use the rest for validation.

4. Convert the data into PyTorch tensors.

5. Set up the optimizer for training.

6. Train the VAE model using the `train()` method. The training process involves minimizing the ELBO (Evidence Lower Bound) loss function, which consists of a Negative Log-Likelihood (NLL) term and a Kullback-Leibler (KL) divergence term. Arguments used in this function include:

    - `x_train`: Input data for training the VAE, a PyTorch tensor representing the VAE input data. 

    - `y_train`: Output data for training the VAE, a PyTorch tensor representing the VAE output data. 

    - `optim`: The optimizer used for training the VAE, a PyTorch optimizer object, such as `torch.optim.Adam`, that specifies the optimization algorithm and its hyperparameters, including the learning rate (`lr`).

    - `x_val`: Input data for validation during training. (optional, default is `None`).

    - `y_val`: Output data for validation during training. (optional, default is `None`).

    - `epochs`: The number of training epochs (epoch: a single pass through the data).

    - `batch_size`: The batch size used during training. If an integer is provided, the same batch size will be used for all epochs. If a list of integers is provided, it should have the same length as the number of epochs, and each value in the list will be used as the batch size for the corresponding epoch. Default is `256`.

    - `w_kl`: The weight of the Kullback-Leibler (KL) divergence term in the ELBO loss function. The KL divergence term encourages the latent distribution to be close to a prior distribution (usually a standard normal distribution). A higher value of `w_kl` will increase the regularization strength on the latent space. Default is `1.0`.
    - `include_errors`, `eps`, `stdof`: when including errors, the loss will be weighted by the errors in the observed amplitudes. A small fixed amount, `eps` can be added to these error estimates to diminish the contribution of the reflections with very small sigF (e.g. around 0.005 after normalization). `stdof` toggles the loss function between Student t (finite `stdof`) and Normal (`stdof=None`).

    **Note:** The VAE class internally keeps track of the training loss (`loss_train`) and its components (NLL and KL divergence) during each batch of training. These values can be accessed after training to monitor the training progress and performance. The `loss_train` attribute of the VAE object will be a list containing the training loss values for each batch during training. The `loss_names` attribute contains the names of the loss components: "Loss", "NLL", and "KL_div". These attributes are updated during training and can be used for analysis or visualization.

7. Save the trained VAE model for future use (optional).

### Code

In [ ]:
# Load the VAE I/O Files Generated
# note that we do not use a prefix since these files are likely to change less frequently than the runs.
vae_input  = np.load(vae_path + 'vae_input.npy')
vae_output = np.load(vae_path + 'vae_output.npy')
vae_sigF   = np.load(vae_path + 'vae_sigF.npy')

In [ ]:
# Sanity checks
print(vae_input.shape)
print(vae_output.shape)
print(vae_sigF.shape)

In [ ]:
# If you want to see the training target in all its glory...
# union = pd.read_pickle(union_path)
# union.iloc[:5,:20]

# or the (normalized) sigF
# sigF = pd.read_pickle(sigF_path)
# sigF.iloc[:5,:20]

# SNR_ratio = sigF.iloc[:500,:500].values.flatten()/union.iloc[:500,:500].values.flatten()

In [ ]:
plt.plot(vae_output[10,:],vae_sigF[10,:],'.',alpha=0.01)
plt.xlabel("Normalized input amplitudes (can be negative!)")
plt.ylabel("Normalized SigF")
plt.show()

#### Setting up the VAE

In [ ]:
%%time
# Specify VAE Parameters; takes ~1 sec
# Only execute once. Otherwise it may throw an error.Restart kernel if this cell throws an error
latent_dimension = latent_dim
train_fraction = train_fraction
epochs = epochs # default: 300

vae = valdo.VAE(n_dim_i = vae_input.shape[1], 
      n_dim_o = vae_output.shape[1], 
      n_dim_latent = latent_dimension, 
      n_hidden_layers = [3, 6], 
      n_hidden_size = 100, 
      activation = torch.relu)

# Randomly select (train_fraction) indices for training
choice = np.random.choice(vae_input.shape[0], int(train_fraction*vae_input.shape[0]), replace=False)    
train_ind = np.zeros(vae_input.shape[0], dtype=bool)
train_ind[choice] = True
test_ind = ~train_ind
print("Size of training set = " + str(np.sum(train_ind)))
print("Size of test set = " + str(np.sum(test_ind)))

# Split the input and output data into training and validation sets
x_train, x_val = vae_input[train_ind], vae_input[test_ind]
y_train, y_val = vae_output[train_ind], vae_output[test_ind]
e_train, e_val = vae_sigF[train_ind], vae_sigF[test_ind] # error estimates for y

# Convert the data to torch tensors
x_train, x_val, y_train, y_val, e_train, e_val = torch.tensor(x_train), torch.tensor(x_val), \
                                                 torch.tensor(y_train), torch.tensor(y_val), \
                                                 torch.tensor(e_train), torch.tensor(e_val)

#### Training the VAE

In [ ]:
%%time
# Set up the optimizer and train the VAE
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
vae.train(x_train, y_train, e_train, optimizer, x_val, y_val, e_val, epochs=epochs, 
          batch_size=100, w_kl=1.0,
          eps=eps,
          include_errors=include_errors,
          stdof=stdof, 
          verbose=True)

In [ ]:
# Save the trained VAE model
vae.save(vae_path + run_prefix + 'trained_vae.pkl')

#### VAE loss traces

The following cells allow us to visualize the loss over epochs.

In [ ]:
vae = valdo.VAE.load(vae_path + run_prefix + 'trained_vae.pkl')
loss_array = np.array(vae.loss_train)
plt.figure(figsize=(5,3.5))
plt.plot(loss_array)
plt.xlabel("Step")
plt.ylabel("Loss term")
plt.grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=[6,8])
ax = axs.reshape(-1)

ax[0].plot(loss_array[:,0], label='Total Loss, Training')
ax[0].plot(loss_array[:,3], label='Total Loss, Validation')
ax[0].set_xlabel("Steps")
ax[0].grid()
ax[0].legend()

ax[1].plot(loss_array[:,1], label='Negative Log Likelihood, Training')
ax[1].plot(loss_array[:,4], label='Negative Log Likelihood, Validation')
ax[1].set_xlabel("Steps")
ax[1].grid()
ax[1].legend()

ax[2].plot(loss_array[:,2], label='KL Divergence, Training')
ax[2].plot(loss_array[:,5], label='KL Divergence, Validation')
ax[2].set_xlabel("Steps")
ax[2].grid()
ax[2].legend()

plt.show()

## Steps 5 & 6: Reconstruction of "Apo" Data & Calculating Difference Maps

In this step, VAE outputs are re-scaled accordingly to recover the original scale, and differences in amplitudes between the original and reconstructed data are calculated. A `recons` and a `diff` column will be created for all datasets.

### Usage

To perform the reconstruction, or re-scaling, the `rescale()` function can be called, providing the necessary arguments:

- `recons_path`: Path to the reconstructed output of the VAE in NumPy format.
- `intersection_path`: Path to the pickle file containing the intersection of all scaled datasets.
- `union_path`: Path to the pickle file containing the union data of all scaled datasets.
- `input_files`: List of input file paths. This list should be in the same order as is in the `vae_input.npy` or `intersection.mtz`.
- `info_path`: Path to the folder containing files with the mean and SD used for standardization previously.
- `output_path`: Path to the folder where the reconstructed data will be saved.
- `amplitude_col`: Column in the MTZ file that contains structure factor amplitudes to calculate the difference column.

### Code
_Loads its inputs, saves its outputs._

In [ ]:
# Load trained VAE
vae = valdo.VAE.load(vae_path + run_prefix + 'trained_vae.pkl')

In [ ]:
# Load input file and create a tensor

vae_input = np.load(vae_path + 'vae_input.npy')
vae_input_tensor = torch.tensor(vae_input)
if bGPU:
    vae_input_tensor = vae_input_tensor.to("cuda:0")

In [ ]:
# Reconstruct the input file via VAE, convert to numpy, and save
recons = vae.reconstruct(vae_input_tensor, ml_recon=ml_recon, repeats=vae_samples)
if vae_samples > 1:
    recons_list = []
    for item in recons:
        recons_list.append(item.detach().cpu().numpy())
    recons_3d = np.array(recons_list)
    recons=np.array([np.mean(recons_3d,axis=0), np.std(recons_3d,axis=0)])
else:
    recons = recons.detach().cpu().numpy()
    
np.save(vae_reconstructed_path + 'recons', recons)

In [ ]:
if vae_samples > 1:
    plt.hist(recons[1].flatten(),20)
    plt.yscale('log')
    plt.show()

In the following step we put the reconstructed amplitudes back on the same scale as the input data. This step can crash the kernel/session if not enough memory is available. In that case, try reducing ncpu below the available number of CPUs.

In [ ]:
%%time
# Re-scale the reconstructed files accordingly and creates the `diff` column
# Function is valdo.preprocessing.rescale

ncpu_temp=10#ncpu/2

with open(os.path.join(vae_path, 'filtered_file_list.pkl'),'rb') as f:
    file_list = pickle.load(f)

print(len(file_list))

if ncpu > 1:
    valdo.preprocessing.rescale_pool(recons_path=vae_reconstructed_path + 'recons.npy', 
                intersection_path=intersection_path, 
                union_path=union_path, 
                input_files=file_list, 
                info_folder=vae_path, 
                output_folder=vae_reconstructed_path,
                amplitude_col=amplitude_scaled_col,
                ncpu=ncpu_temp)
else:
    valdo.preprocessing.rescale(recons_path=vae_reconstructed_path + 'recons.npy', 
                intersection_path=intersection_path, 
                union_path=union_path, 
                input_files=file_list, 
                info_folder=vae_path, 
                output_folder=vae_reconstructed_path,
                amplitude_col=amplitude_scaled_col)

## Steps 7 & 8: Gaussian Blurring & Searching for Blobs

In this step, we aim to identify significant changes in electron density caused by ligand binding to a protein. By taking the absolute value of the electron density difference maps and applying Gaussian blurring, a new map is created with merged positive electron density blobs. The blurring process attempts to reduce noise. Blobs are then identified and characterized above a specified contour level and volume threshold.

### Usage

To generate blobs from electron density maps, call the `generate_blobs()` function, which takes electron density map files and corresponding refined protein models as inputs. The function preprocesses the maps and identifies blobs above a specified contour level and volume threshold (the volume threshold is the default set by `gemmi`). The output is a DataFrame containing statistics for each identified blob, including peak value, score, centroid coordinates, volume, and radius. 

This function can be called with the following arguments:

- `input_files`: List of input file paths.
- `model_path`: Path to the folder containing the refined models for each dataset (pdb format).
- `diff_col`: Name of the column representing diffraction values in the input MTZ files.
- `phase_col`: Name of the column representing phase values in the input MTZ files.
- `output_path`: Path to the output folder where the blob statistics DataFrame will be saved.
- `cutoff`: Blob cutoff value. Blobs with values below this cutoff will be ignored (optional, default is `5`).
- `negate`: Whether to negate the blob statistics (optional, default is `False`). Use True if there is interest in both positive and negative peaks, which is not typically of interest here due to the absolute value function applied to the map.
- `sample_rate`: Sample rate for generating the grid in the FFT process (optional, default is `3`).

### Add Phases from Apo Refinement Code

The following cells **add phases** to our newly reconstructed datasets. These phases are copied from `refined_path` which were generated via PHENIX in 0.3.3

In [ ]:
%%time
print(phasing_path)
print(vae_reconstructed_with_phases_path)

# List of reconstructed mtz files without phases to add phases to
file_list = glob.glob(vae_reconstructed_path + "*.mtz")
# file_list = file_list[:10]
file_list_w_phases = file_list.copy()
print("\nWorking with " + str(len(file_list)) + " MTZs containing reconstructed (apo-like) amplitudes.\n")

#### Linking reconstructed amplitude MTZs to phases MTZs
We need to know how to link the MTZ with VAE-reconstructed amplitudes to refinement MTZs containing phases. `valdo.helper.find_phase_file()` can try to do so, but it is best to provide a user-written parser to make sure this happens correctly:


In [ ]:
print(phasing_path)
print(apo_phases_parser(file_list[0]))
print(valdo.helper.find_phase_file(file_list[0], phasing_path, parser=apo_phases_parser))

#### adding phases

In [ ]:
if ncpu > 1:
    no_phases_files = valdo.helper.add_phases_pool(
        file_list, 
        phasing_path, 
        vae_reconstructed_with_phases_path, 
        phase_2FOFC_col_out=phase_2FOFC_col_out, phase_FOFC_col_out=phase_FOFC_col_out,
        phase_2FOFC_col_in =phase_2FOFC_col_in,  phase_FOFC_col_in =phase_FOFC_col_in,
        prefix=run_prefix, 
        parser=apo_phases_parser,
        ncpu=ncpu)
    print("Done. No phases found for " + str(len(no_phases_files)) + " starting MTZs.")
else:
    no_phases_files = valdo.helper.add_phases(
        file_list, 
        phasing_path, 
        vae_reconstructed_with_phases_path, 
        phase_2FOFC_col_out=phase_2FOFC_col_out, phase_FOFC_col_out=phase_FOFC_col_out,
        phase_2FOFC_col_in =phase_2FOFC_col_in,  phase_FOFC_col_in =phase_FOFC_col_in,
        parser=apo_phases_parser
    )
    print("Done. No phases found for " + str(len(no_phases_files)) + " starting MTZs.")

In the case of dimple refinement for PTP-1B the ones that are missing are the ones for which both indexing solutions gave a poor CC.

##### Add weights

In [ ]:
%%time
file_list = glob.glob(vae_reconstructed_with_phases_path + "*.mtz")
print("Working with " + str(len(file_list)) + " MTZs containing reconstructed (apo-like) amplitudes.")

result = valdo.helper.add_weights(file_list, 
                                  sigF_col="SIGFP", 
                                  diff_col="diff",
                                  sigdF_pct=w_pcts[0], 
                                  absdF_pct=w_pcts[1],
                                  redo=True,
                                  low_res_cutoff=low_res_cutoff,
                                  ncpu=ncpu)
print("Added weights and weighted differences to " + str(sum(result)) + " MTZ files.")

In [ ]:
%%time
file_list = glob.glob(vae_reconstructed_with_phases_path + "*.mtz")
result=valdo.helper.extrapolate(file_list, \
                                F_col=amplitude_scaled_col, \
                                sigF_col=error_scaled_col, \
                                recons_col="recons", \
                                sigF_recons_col="SIG_recons", \
                                diff_col="diff", \
                                wt_col="WT", \
                                redo=True, \
                                weighted=False, \
                                extrapolate_factors=[4,8,16], \
                                ncpu=ncpu)
print("Calculated extrapolated structure factor amplitudes for " + str(sum(result)) + " MTZ files.\n")

### Gaussian Blurring Code

The following two cells complete gaussian blurring and blob searching. For the blurring, by default, he radius is set to `5A` with `sigma = 5/3`.

In [ ]:
# List of reconstructed mtz files (with phases) to identify blobs in
# tmp="/n/holyscratch01/hekstra_lab/dhekstra/valdo-tests/pipeline_run2/vae/reconstructed_w_phases/"

# usually vae_reconstructed_with_phases_path rather than tmp
file_list = glob.glob(vae_reconstructed_with_phases_path + "*mtz")
print("Retrieving " + str(len(file_list)) + " files for blob analysis.")

The following takes about 0.3 s/dataset on a single CPU for PTP1B and will scale with the number of voxels.

In [ ]:
%%time
# Function in valdo.blobs that generates a list of blobs

if ncpu>1:
    valdo.blobs.generate_blobs_pool(
        input_files=file_list, 
        model_folder=refined_path, 
        diff_col='WDF', # DEFAULT is "diff"!!!!
        phase_col='refine_PH2FOFCWT', 
        output_folder=blob_path, 
        prefix=run_prefix,
        cutoff=blob_sig_cutoff,
        radius_in_A=radius_in_A,
        ncpu=ncpu)
else:
    valdo.blobs.generate_blobs(
        input_files=file_list, 
        model_folder=refined_path, 
        diff_col='WDF', 
        phase_col='refine_PH2FOFCWT', 
        output_folder=blob_path, 
        prefix=run_prefix,
        cutoff=blob_sig_cutoff,
        radius_in_A=radius_in_A)

## Step 9: Identifying Events

In this final step, the highest scoring blobs returned in the previous step can be analyzed individually. If the blob is plausibly a ligand, refinement with a ligand may be completed to determine whether or not the blob can be considered a "hit."

Blobs that are returned can be related to various other events, not just ligand binding. Examples may include ligand-induced conformational change (which would still indicate the presence of a ligand) or various other unrelated conformational changes, such as radiation damage or, for example for PTP1B, cysteine oxidation.

In the following example, we have also included the evaluation of our method, via AUC, in this section.

### Tagging Blobs

In this section, we tag and filter the blobs. We remove...

- blobs that are duplicates (we occassionally have duplicate blobs due to an issue with gemmi's ASU mask)
- blobs associated with the oxidation of `cys215`
- blobs that belong to low quality samples (high r factors in refinement)
- blobs that belong to samples with inconsistent data (in particular, Helen Ginn lists a few samples as hits without including a ligand in their bound state).

We also label blobs that are close to ligand positions known from a previous PanDDA analysis.

In [ ]:
blob_df = pd.read_pickle(blob_path + run_prefix + 'blob_stats.pkl')
blob_df.head(5)

Tag Samples that are known to contain a ligand (``Bound''): (1 if bound, 0 otherwise). Based on a previous PanDDA analysis. You can skip this if no such analysis!

In [ ]:
blob_df["bound"] = 0

In the case of PTP-1B, the catalytic cysteine, residue 215, is prone to oxidation with levels varying among datasets. To get rid of these false positives, we tag Samples within 3A of a Cys215 Atom (1 if within, 0 otherwise). You can use/adapt `valdo.tag.tag_blobs_around_seqid` more generally to omit certain regions.

In [ ]:
%%time
#with one CPU, this takes about 40 sec

# blob_df = valdo.tag.tag_blobs_around_seqid(blob_df, 
#                                            refined_path, 
#                                            radius=5, # increased from 3
#                                            tag='cys215', 
#                                            focal_seqid=215, 
#                                            ncpu=ncpu)

In the case of PTP-1B, we already had an analysis by Keedy et al in hand using PanDDA. Here we tag samples within `r` of a known LIG atom (1 if yes, 0 otherwise). Fortunately, in all their bound models, the ligand "residue name" was LIG.

In [ ]:
# %%time
# blob_df = valdo.tag.tag_lig_blobs(blob_df, 
#                                   bound_models_standardized_path,
#                                   ncpu=ncpu)

We use GEMMI under the hood to restrict blobs to a single asymmetric unit. Sometimes, however, we still double-include blobs if they are not in the exact same position. Here we tag Blobs that are Duplicates of Other Blobs (a.k.a. Patch for Gemmi's ASU Mask Issue).

In [ ]:
%%time
# Identifies all possible cartesian coordinates after symmetry operations
# blob_df[['fractional', 'all_possible_frac', 'all_possible_cart']] = blob_df.apply(determine_locations, args=(vae_reconstructed_with_phases_path,), axis=1)
blob_df = valdo.tag.determine_locations(blob_df, vae_reconstructed_with_phases_path, ncpu=ncpu)
# blob_df.info()

In [ ]:
%%time 
# <1 sec
# Marks blobs as duplicates if they are within 1A of another blob in the same sample
blob_df = valdo.tag.mark_duplicates(blob_df)

Let's look at some summary statistics so far.

In [ ]:
blob_df.head()

Tag Blobs Belonging to Samples with Low cc after scalings

In [ ]:
# In this case, no blobs are removed because we already did so before
metrics_df = pd.read_pickle(scaled_path + run_prefix + "scaling_metrics.pkl")

low_end_corr = metrics_df.loc[metrics_df['end_corr'] < min_cc_ref, 'file'].tolist()
blob_df['low_end_corr'] = blob_df['sample'].isin(low_end_corr)
print("Number of blobs with end_corr < " + str(min_cc_ref) + " = " +str(blob_df['low_end_corr'].sum()))

# Should be 0 since we did this already

In [ ]:
blob_df.to_pickle(blob_path + run_prefix + 'blob_stats_tagged.pkl')

### Filter Blobs 
Remove blobs near Cys215, with high R_free, duplicates, and more as specified below.

In [ ]:
blob_df = pd.read_pickle(blob_path + run_prefix +'blob_stats_tagged.pkl')

In [ ]:
blob_df['sample_id'] = blob_df["sample"].apply(lambda x: x.split("_")[0])

In [ ]:
# Remove Cys215 related blobs, and duplicates

blob_df = blob_df[(blob_df['duplicate']==0)]   

Let's look at the filtered set of blobs:

In [ ]:
blob_df.describe()

In [ ]:
blob_df.to_pickle(blob_path + run_prefix+'filtered_blob_stats_tagged.pkl')

In [ ]:
sorted_blobs = blob_df.sort_values(by="peakz",axis=0,ascending=False)
sorted_blobs.iloc[0:40,:][["sample","peakz","score","bound"]]